In [1]:
import pandas as pd
from math import fsum
from sklearn.metrics import classification_report

In [2]:
# We get all our predictions
hatebert_in = pd.read_csv('data/hateBERT_indomain.csv')
hatebert_cross = pd.read_csv('data/hateBERT_crossdomain.csv')

bert_in = pd.read_csv('data/BERT_indomain.csv')
bert_cross = pd.read_csv('data/BERT_crossdomain.csv')

svm_in = pd.read_csv('data/SVM_indomain.csv').drop(['Unnamed: 0'], axis=1)
svm_cross = pd.read_csv('data/SVM_crossdomain.csv').drop(['Unnamed: 0'], axis=1)

# Dataframes to write results in
test_indomain = pd.read_csv('data/olid-test.csv')
test_crossdomain = pd.read_csv('data/olid-test.csv')

In [6]:
# We implement soft voting, the class with highest summed probability from each classifier is picked for each instance
def soft_voting(df1, df2 ,df3):
    ensemble_predictions = []
    for p1, p2, p3,\
    pn1, pn2, pn3\
    in zip(df1.p_pos, df2.p_pos, df3.p_pos,\
          df1.p_neg, df2.p_neg, df3.p_neg):
        if fsum([p1, p2, p3]) >= fsum([pn1, pn2, pn3]):
            ensemble_predictions.append(1)
        else:
            ensemble_predictions.append(0)
    return ensemble_predictions    

In [4]:
# we get results for both in domain and cross domain
test_indomain['predictions'] = soft_voting(hatebert_in, bert_in, svm_in)

test_crossdomain['predictions'] = soft_voting(hatebert_cross, bert_cross, svm_cross)

In [5]:
# Writing results to disk
test_indomain.to_csv('data/SOFTVOTING_indomain.csv', index=False)

test_crossdomain.to_csv('data/SOFTVOTING_crossdomain.csv', index=False)